# User Accounts and Privileges

## User Account Basics

- Restricting to specific databases
In order to limit the lena_stankoska@lena_stanoska_home user account to the rookery databse, we would have to do something like this:

```sql
GRANT SELECT ON rookery.* TO 'lena_stankoska'@;lena_stankoska_home';

SHOW GRANTS FOR 'lena_stankoska'@'lena_stankoska_home';
```

### Administrative User Accounts

Earlier, I mentioned that we need to create three administrative accounts for Lena to use in performing her duties as a database administrator from the localhost: admin_backup, admin_restore, and admin_import. These are common administrative user accounts that you may need to create and use. You'll use them in examples and exercises in Chapter 14 and Chapter 15. In this section, we'll create these administrative user accounts and loook at the privileges needed for them, as well as another one for granting privileges to other user accounts.


## User Account for Making Backups

The admin_backup user account will be used with the mysqldump utility to make backups of the rookery and birdwatchers databases. This is covered in Chapter 14. Just a few privileges are needed to accomplish these tasks:

- At a minimum, it will need the SELECT privilege to read our two databases. You should limit an administrative account to the databases it needs to backup. In particular, you should not let it have SELECT privileges for the mysql database, because that contains user passwords.
- To lock the tablees when making a backup, the LOCK TABLES privilieges is required. 
- If a database contains views and triggers, which we didn't cover in this book, the user account will need the SHOW VIEW and TRIGGER privileges, respectively.

### User Account for Restoring Backups

Although you could create one administrative user account for both making backups and restoring them, you might want to use separate user accounts for those tasks. The main reason is that the task of making backups is usually one handled by scripts that run automatically. But the task of restoring data is generally run manually and can overwrite or destroy data on a live server. You might not want the user account with those privileges to be the same one for which you use in a script containing its password. For our examples in this chapter, let's give the admin_restore@localhost user account the privileges needed for restoring data to our databases:

### User Account to Grant Privileges

## User Roles

# Backing up and Restoring Database

- mysqldump 
It's included with both servers and it costs you nothing. You probably already have it installed on your server. Best of all, it doesn't require you to shut down MySQL services to make a backup, although you might restrict access to the backup utility for better consistency of data. There are other backup utilities, some with a GUI and some that are more comprehensive. You can learn about other types of backups and tools in the book MySQL Troubleshooting. 

# Backing Up all databases

The simplest way to make a backup is to dump all of the databases with all of the tables and their data. You can do this easily with mysqldump. Try executing something like the following at the command line on your server, using the administrative user you created in Ch13.

```sql
mysqldump --user=admin_backup \
    --password --lock-all-tables
    --all-databases > /datal/backups/all-dbs.sql
```

```
When making backups of InnoDB or other transactional tables with mysqldump, it’s best to include the --single-transaction option. This will keep the data more consistent. It won’t change between the tables until the dump is finished. However, that option will cancel the --lock-tables option. This means that a backup of MyISAM tables in the same database could be inconsistent. You can avoid this po‐ tential problem by either using the same storage engine for all of the tables in a database, or making separate backups of InnoDB tables and MyISAM tables.
``` 

Then you can attempt to merge the old and new data together. Depending on the situation, you might be able to do this by changing the INSERT . to a REPLACE statement. Another method would be to remove the DROP TABLE statement and change the anme of CREATE TABLE statement that follows to create a new table name. We'll cover such techniques later in this chapter in ~/

 The IF EXISTS option ensures that a restore will drop the table only if it exist. If this statement was omitted, a restore would probably try to run the statement when the table didn't exist, and thus generate an error that could abort the restore.
 
After the DROP TABLE statement, there are more conditional SQL statements for variables related to the table and the client. These are followed by the CREATE TABLE statement, which matches the results of a SHOW CREATE TABLE statement for the table. This section ends by returning the variable changed to its previous setting.

Now the bird_families table is ready for the data. The nex set of entries in the dump file are:

After the comment appears a LOCK TABLES statement to lock the bird_families table. It includes the WRITE option so that the data in the table cannot be changed during the restoration of the table. Users can't read the table either. Another thought may have occurred to you noew: mysqldump is write-locking tables one at a time, as needed. That may be what you want, making other tables available for reading and writing when they're not being dumped. However, this may cause a problem with the consistency of the data.

For example, suppose during backup is at the ping where is has preserved the contents of the humans table but not the bird_sightings table in the birdwatchers databse. At this poiint, you decided to delete someone from the humans table along with entries in the bird_sightings table. If you were later to restore the entire birdwatchers database, you would have an entries in the bird_sightings table for a person who isn't listed in the humans table.

If a database isn't very active, the previous scenario is unlikely. However, if you want to be assured of the consistency of your data, when executing the mysqldump utility, you could add the --lock-tables option. This locks all tables in a database before backing it up, and leaves them locked until the backup of the database is completed.

## Backung up specific databases

Before we concerned ourselves so much with the contents of the dump file, we were experimenting with making backups, learning how to back up all databases on the server. However, you may want to export only one databse, or only specific ones. Let's see how to do that.

To export only one database and not all, instead of uning the --all-databses option, use the --databases option followed by the name of the database. Try making a back up of just the rookey databse by entering the following on your server from the command line:

```sql
mysqldump --user=admin_backup --password --lock-tables \
--verbose --databases rookery > rookery.sql
```  

This is basically the same as the example that dumped all of the databases, except that we've specified the database to be exported, rookery. As mentioned before, you may want to make separate backups of databases to reduce the load on a busy server and to naje restiration more manageable. Incidentally, if for some reason you want to make a backup of a database's schema without the data, you can use the --no-data option. The command would then dump only the database and table schemas and no the rows of data.

You may have noticed in the previous example that we added the --verbose option. This option instructs the utility to display messages regarding each major step in the process of quering the database and creating the dump file. For our database, running this command produces messages like this:

## Creating Backup Scripts

To automate many aspects of making backups of databses, it's useful to create a set of scripts that will exectue the mysqldump for the databses you want with the settings that you prefer. It's not too difficult to do this. You don't need to be very advanced in programming if you want to do only a few simple things, such as varying the output slightly each time.

Let's use the problem presented at the end of the previous section for an example backup script. The solution is to change the name of the dump file each day to include the current date so that there will a unique dump file for each day. Here's an example of a very simple shell script that may ne run on a Linux or Mac system to do this:

```sql
msqldump --user=admin_backup --password --lock-tables \
 --databases rookery --tables birds > birds-humnas.sql
 ``` 

## Restoring Backups


If data is lost in MySQL, but you've been using mysqldump to make regular backups of the data, you can use the dump files to restore the data. This is the point of the backups, after all. Restoring a dump file made with mysqldump is just a matter of using the mysql client to execute all of the SQL statements contained in the dump file. You can restore all of the databases, a single databse, individual tables, or even specific rows of data. We'll cover all of these in this section.

### Restoring a Database

Let's look at how to restore an entire database. To be safe, as part of experimenting, we'll make a fresh backup of the rookery databse and then resotre it. Execute the following from the command line on your server:

```sql
mysqldup --user=admin_backup --password --lock-tables \
--databases rookery > rookery.sql
``` 

Before proceeding, check he contents of the dump file. Make sure it contains the SQL statements for restoring the rookery database. If everything looks OK, delete the rookery database from the server. This may seem scary, but you just made a good back-up.

There will come a time when a database is deleted or corrupted unintentionally. So it's better to develop confidence in your abilirt to restore a database with a test database like rookery. To get rid of the database, you can exectue the following from the command line.

### Modifying a dump file

You can make a copy of the rookery.sql dump file, open the copy with a plain-text editor, and delete the sections that create the other tables. Leave in the opening and closing lines that set the variables, as well as the section for the conservation_status table. A similar method would be to open the dump file in a text editor and then copy and/paste the parts you neeed into a new text document: the opening and 

```sql
 -- MySQL dump 10.14  Distrib 5.5.39-MariaDB, for Linux (i686)
    --
    -- Host: localhost    Database: rookery
    -- ------------------------------------------------------
    -- Server version       5.5.39-MariaDB
    /*!40101 SET @OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT */;
    /*!40101 SET @OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS */;
    /*!40101 SET @OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION */;
    /*!40101 SET NAMES utf8 */;
    /*!40103 SET @OLD_TIME_ZONE=@@TIME_ZONE */;
    /*!40103 SET TIME_ZONE='+00:00' */;
    /*!40014 SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0 */;
    /*!40014 SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS,FOREIGN_KEY...=0*/;
    /*!40101 SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='NO_AUTO_VALUE_ON_ZERO' */;
    /*!40111 SET @OLD_SQL_NOTES=@@SQL_NOTES, SQL_NOTES=0 */;
    --
    -- Current Database: `rookery`
    --
    CREATE DATABASE /*!32312 IF NOT EXISTS*/ `rookery`
    /*!40100 DEFAULT CHARACTER SET latin1 COLLATE latin1_bin */;
USE `rookery`; -- [ snip ]
    --
    -- Table structure for table `conservation_status`
    --
    DROP TABLE IF EXISTS `conservation_status`;
    /*!40101 SET @saved_cs_client     = @@character_set_client */;
    /*!40101 SET character_set_client = utf8 */;
    CREATE TABLE `conservation_status` (
      `conservation_status_id` int(11) NOT NULL AUTO_INCREMENT,
      `conservation_category` char(10) COLLATE latin1_bin DEFAULT NULL,
      `conservation_state` char(25) COLLATE latin1_bin DEFAULT NULL,
      PRIMARY KEY (`conservation_status_id`)
    ) ENGINE=MyISAM AUTO_INCREMENT=10
      DEFAULT CHARSET=latin1 COLLATE=latin1_bin;
    /*!40101 SET character_set_client = @saved_cs_client */;
    --
    -- Dumping data for table `conservation_status`
 294
| Chapter 14: Backing Up and Restoring Databases
--
    LOCK TABLES `conservation_status` WRITE;
    /*!40000 ALTER TABLE `conservation_status` DISABLE KEYS */;
    INSERT INTO `conservation_status` VALUES
    (1,'Extinct','Extinct'),
    (2,'Extinct','Extinct in Wild'),
    (3,'Threatened','Critically Endangered'),
    (4,'Threatened','Endangered'),
    (5,'Threatened','Vulnerable'),
    (6,'Lower Risk','Conservation Dependent'),
    (7,'Lower Risk','Near Threatened'),
    (8,'Lower Risk','Least Concern'),
    (9,NULL,'Unknown');
/*!40000 ALTER TABLE `conservation_status` ENABLE KEYS */; UNLOCK TABLES;
-- [ snip ]
/*!40103 SET TIME_ZONE=@OLD_TIME_ZONE */;
    /*!40101 SET SQL_MODE=@OLD_SQL_MODE */;
    /*!40014 SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS */;
    /*!40014 SET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS */;
    /*!40101 SET CHARACTER_SET_CLIENT=@OLD_CHARACTER_SET_CLIENT */;
    /*!40101 SET CHARACTER_SET_RESULTS=@OLD_CHARACTER_SET_RESULTS */;
    /*!40101 SET COLLATION_CONNECTION=@OLD_COLLATION_CONNECTION */;
    /*!40111 SET SQL_NOTES=@OLD_SQL_NOTES */;
    -- Dump completed on 2014-09-15  6:48:27
```

### Using a limited user account

A simple way to restore only one table is to create a temporary user account that has only privileges for the table you want to restore. When you run the dump file, the SQL statements for other tables will fail and not be executed - only the table for which the user account has privileges will be restored. To create such a user account, you need the GRANT OPTION privilege. AS root, you will have thay privilege. Let's go through the steps involved in this method, using the previous example in which we want to restore the conservation_status table.

Before you start to restore your data, delete the conservation_status table and change some data in one of the tables so that you can see how well this method works. You can run something like the following from the command line, using hte admin_boss user account you should have created in the Chapter 13 exercises:

That should delete the conservation_status table. To test our restore, we've also added a row to the birds table, which we want to make sure has not been lost when we do our restore. The last statement returns the bird-id for the row inserted. Log into MySQL and verify that the conservation_status table has been deleted and use the SELECT statement to view the row inserted into birds, where the bird_id equals the number you were given when you executed the command. If everything looks as it should, you're ready to proceed.

## Restoring Only Rows or Columns

You'll rarely nedd to restore an entire database or even an entire table. It's not often that a database or a table is dorpped, or the the data in all of the orws in a table are changed accidentally. It's more commnon that someone deltes a single row in a table or data in a single column and can't undo what they did. In such a situation, if the table has many other rows that were changed correctly since the last backup was made, you wouldn't want to restore the whole table to fix one small mistake. Instead, you will want to restore only one row or column.

## Recovering from a Binary Log

To do point-in-time recoveries, you will have to enable the binary logs. YOu can't wait until you need them; you have to enable the binary logs before a problem occues. To check that it's enabled, execute the following from the mysql client.

```sql
SHOW BINARY LOGS;
``` 


```bash
[mysqld]
log-ibin
binlog-ignore-db=mysql
``` 

```sql
show master status;
``` 

```sql
mysql --user=admin_maintenace --password -- database=rookery < birds-simple.sql
mysql --user=root --password --silent \
  --execute="SELETE COUNT(*) AS '' FROM rookery.birds_simple;"
 ```

```sql
# at 1258707
    #140916 13:10:24 server id 1 end_log_pos 1258778
    Query thread_id=382 exec_time=0 error_code=0
    SET TIMESTAMP=1410887424/*!*/;
    SET @@session.sql_mode=0/*!*/;
BEGIN /*!*/;
    # at 1258778
    #140916 13:10:24 server id 1 end_log_pos 1258900
    Query thread_id=382 exec_time=0 error_code=0
    use `rookery`/*!*/;
    SET TIMESTAMP=1410887424/*!*/;
    DELETE FROM birds_simple WHERE common_name LIKE '%Blue%'
    /*!*/;
    # at 1258900
    #140916 13:10:24 server id 1 end_log_pos 1258927 Xid = 45248
    COMMIT/*!*/;
    ...
    # at 1284668
    #140916 13:10:28 server id 1 end_log_pos 1284739
    Query thread_id=382 exec_time=0 error_code=0
    SET TIMESTAMP=1410887428/*!*/;
    SET @@session.sql_mode=0/*!*/;
    BEGIN
    /*!*/;
    # at 1284739
    #140916 13:10:28 server id 1 end_log_pos 1284862
    Query thread_id=382 exec_time=0 error_code=0
    SET TIMESTAMP=1410887428/*!*/;
    DELETE FROM birds_simple WHERE common_name LIKE '%Green%'
    /*!*/;
    # at 1284862
    #140916 13:10:28 server id 1 end_log_pos 1284889 Xid = 45553
    COMMIT/*!*/;
```

```sql
mysqlbinlog --database=rookery --stop-position="1258707" \
/data/mysql/mysqlresource-bin.000002 \
mysql --user=admin_maintenance --password
```

# Bulk Importing Data

You might be asked one day to create a MySQL or MariaDB database that will replace an existing database that uses a different database system - or some other format use for storing data. Or you might be asked to take the data from an application that was not designed for databases, like a spreadsheet. So that you don't have to manually enter the data, there are ways you can import it. This chpater explains how to bulg import data into a database.

When using other applications, export the data from the source application to a format that MySQL can read, such as a text file containing data separated by particular characters. If you're given a large amount of data to import, hopefully it will already be well organized and in a data text file. Then you can use the LOAD DATA INFILE statement to import the data.

This isn'n an overly difficult task, but the processing of large amounts of data can be intimidating the first time. It can be a barrier to migrating data to MySQL and MariaDB. There are many nuances to consider for a clean import, which is especially important if you want to automate the process. There may also be restraints to consider when importing data onto a server provided be a web hosting company. We'll cover all of these in this chapter.

# Preparing to Import 

To immport data into MySQL or MariaDB, the data needs to be in a compatible format. Both database systems will accept a simple text file in which the values are delimited in some way. The easiet way to deal with incompatible data in any format is to load it in its original software and to export it to a delimited text file. Most applications can do this. They will usually separate field values by commas and separate records by hard returs, putting each row on a separate line. Some applications will allow you to set the delimiters to your choice. If that's an option, you might use the bar to separate fields because it won't typically be used within the data, and separate records with new-line.

```sql
 LOAD DATA INFIlE '/tmp/Clements-Checklist-6.90final.csv'
 INTO TABLE rookery.clements_list_import
 FIELDS TERMINATED BY ',';
 ``` 